<a href="https://colab.research.google.com/github/Sanket1909/nextwordpro/blob/main/Next_Word_Prediction(Group_5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers, models

# Preprocessing data

In [ ]:
# Read the file
fp = open('dataset.txt','r')
data = fp.read().splitlines()        
fp.close()

# Encode the data
tokens = text.Tokenizer()
tokens.fit_on_texts(data)
data_sequences = tokens.texts_to_sequences(data)
vocab_size = len(tokens.word_counts) + 1

# generate the sequence
seq_list = list()
for item in data_sequences:
    l = len(item)
    for id in range(1, l):
        seq_list.append(item[: id+1])
        
max_length = max([len(seq) for seq in seq_list])
data_sequences_matrix = sequence.pad_sequences(seq_list, maxlen = max_length, padding = 'pre')
data_sequences_matrix = np.array(data_sequences_matrix)

# separate input data X and corresponding output y
X = data_sequences_matrix[:, :-1]
y = data_sequences_matrix[:, -1]
y = to_categorical(y, num_classes = vocab_size)

# RNN

In [ ]:
lstm_model = models.Sequential()
lstm_model.add(layers.Input(shape = [max_length-1]))
lstm_model.add(layers.Embedding(vocab_size, 10, input_length = max_length-1))
lstm_model.add(layers.LSTM(50))
lstm_model.add(layers.Dropout(0.1))               
lstm_model.add(layers.Dense(vocab_size, activation = 'softmax'))
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 10)            7420      
                                                                 
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 742)               37842     
                                                                 
Total params: 57,462
Trainable params: 57,462
Non-trainable params: 0
_________________________________________________________________


# Compiling and fitting on the training data

In [ ]:
lstm_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = lstm_model.fit(X, y, batch_size = 2, epochs = 50)

Epoch 1/50
927/927 [==============================] - 8s 7ms/step - loss: 6.2121 - accuracy: 0.0426
Epoch 2/50
927/927 [==============================] - 6s 6ms/step - loss: 5.7726 - accuracy: 0.0496
Epoch 3/50
927/927 [==============================] - 6s 7ms/step - loss: 5.5914 - accuracy: 0.0496
Epoch 4/50
927/927 [==============================] - 6s 6ms/step - loss: 5.4787 - accuracy: 0.0496
Epoch 5/50
927/927 [==============================] - 6s 7ms/step - loss: 5.3537 - accuracy: 0.0550
Epoch 6/50
927/927 [==============================] - 6s 6ms/step - loss: 5.1972 - accuracy: 0.0588
Epoch 7/50
927/927 [==============================] - 6s 7ms/step - loss: 5.0098 - accuracy: 0.0691
Epoch 8/50
927/927 [==============================] - 6s 6ms/step - loss: 4.8291 - accuracy: 0.0826
Epoch 9/50
927/927 [==============================] - 6s 7ms/step - loss: 4.6414 - accuracy: 0.0885
Epoch 10/50
927/927 [==============================] - 6s 6ms/step - loss: 4.4764 - accuracy: 0.1004

# Saving the model

In [ ]:
lstm_model.save('model.h5')

# Making Predictions

In [ ]:
idx2word = {v:k for k,v in tokens.word_index.items()}
new_model = models.load_model('./saved_model/')

# function to make predictions, it takes text as input and predict *num_words* possible after this text
def predict_words(text, num_words):
    encoded_data = tokens.texts_to_sequences([text])[0]
    padded_data = sequence.pad_sequences([encoded_data], maxlen = max_length - 1, padding = 'pre')
    y_preds = new_model.predict(padded_data)
    y_preds = np.argsort(-y_preds)
    y_preds = y_preds[0][:num_words]
    possible_words = [idx2word[item] for item in y_preds]
    print(text, possible_words)

In [ ]:
predict_words("Where is", 3)
predict_words("Find a", 3)
predict_words("What are", 3)
predict_words("I am not", 3)
predict_words("How to", 3)

In [ ]:
# Saving the model
import os
lstm_model.save('lstm_model.h5')
